In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. Setup Models
# Small model for classification/extraction tasks
fast_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Large model for detailed content generation
powerful_model = ChatOpenAI(model="gpt-4o", temperature=0.7)

# 2. Step 1: Classification (Small Model)
classify_prompt = ChatPromptTemplate.from_template(
    "Classify the following request as 'Creative' or 'Technical': {input}"
)
classification_chain = classify_prompt | fast_model | StrOutputParser()

# 3. Step 2: Content Generation (Large Model)
creative_prompt = ChatPromptTemplate.from_template(
    "Write a detailed, beautiful story about: {input}"
)
technical_prompt = ChatPromptTemplate.from_template(
    "Provide a rigorous, technical explanation of: {input}"
)

# 4. Step 3: Verification (Quality Check Model)
verify_prompt = ChatPromptTemplate.from_template(
    "Review this content for accuracy and tone. If it is good, say 'PASSED'. \n\nContent: {content}"
)
verification_chain = verify_prompt | fast_model | StrOutputParser()

# --- Execution Workflow ---
user_input = "The physics of black holes"

# Run Classification
category = classification_chain.invoke({"input": user_input})
print(f"--- Intent Classified as: {category} ---")

# Route to Powerful Model based on category
if "Technical" in category:
    content = (technical_prompt | powerful_model | StrOutputParser()).invoke(
        {"input": user_input}
    )
else:
    content = (creative_prompt | powerful_model | StrOutputParser()).invoke(
        {"input": user_input}
    )

# Run Verification
check = verification_chain.invoke({"content": content})
print(f"--- Quality Check: {check} ---")
print(f"\nFinal Content Sample:\n{content[:200]}...")

--- Intent Classified as: The request "The physics of black holes" is classified as 'Technical'. ---
--- Quality Check: PASSED ---

Final Content Sample:
Black holes are among the most fascinating and complex objects in the universe, characterized by their intense gravitational fields from which nothing can escape, not even light. The study of black ho...
